In [3]:
import os
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm import tqdm

sys.path.append('./input/')

In [4]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

ModuleNotFoundError: No module named 'greatbarrierreef.competition'

In [ ]:
model = torch.hub.load('/home/xyb/Project/Git/yolov5/runs/train/GBR-Starfish-Detect', 
                       'custom', 
                       path='/home/xyb/Project/Git/yolov5/runs/train/GBR-Starfish-Detect/weights/best.pt',
                       source='local',
                       force_reload=True)  # local repo
model.conf = 0.01

In [ ]:
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    anno = ''
    r = model(img, size=3600, augment=True)
    if r.pandas().xyxy[0].shape[0] == 0:
        anno = ''
    else:
        for idx, row in r.pandas().xyxy[0].iterrows():
            if row.confidence > 0.15:
                anno += '{} {} {} {} {} '.format(row.confidence, int(row.xmin), int(row.ymin), int(row.xmax-row.xmin), int(row.ymax-row.ymin))
#                 pred.append([row.confidence, row.xmin, row.ymin, row.xmax-row.xmin, row.ymax-row.ymin])
    pred_df['annotations'] = anno.strip(' ')
    env.predict(pred_df)